### We will use a Keras deep learning model to predict churn for bank customers.

### First, let's set up our environment.

In [47]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_m

import keras
from keras.models import Sequential
from keras.layers import Dense

import jb_helper_functions_prep
from jb_helper_functions_prep import create_enc

In [34]:
df = pd.read_csv('Churn_Modelling.csv')

In [35]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [36]:
df = create_enc(df, ['Geography', 'Gender'])
df = df[['CreditScore', 'Geography_enc',
       'Gender_enc', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited']]

In [38]:
df.head(3)

,CreditScore,Geography_enc,Gender_enc,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1


### For X, we drop the first few columns that are not needed.  We also encoded Geograpy and Gender to be able used as features.  We'll also need to standardize a few columns to better be used in the Keras model.

In [39]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [40]:
X.shape, y.shape

((10000, 10), (10000,))

In [41]:
X[:5]

array([[6.1900000e+02, 0.0000000e+00, 0.0000000e+00, 4.2000000e+01,
        2.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [6.0800000e+02, 2.0000000e+00, 0.0000000e+00, 4.1000000e+01,
        1.0000000e+00, 8.3807860e+04, 1.0000000e+00, 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [5.0200000e+02, 0.0000000e+00, 0.0000000e+00, 4.2000000e+01,
        8.0000000e+00, 1.5966080e+05, 3.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       [6.9900000e+02, 0.0000000e+00, 0.0000000e+00, 3.9000000e+01,
        1.0000000e+00, 0.0000000e+00, 2.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 9.3826630e+04],
       [8.5000000e+02, 2.0000000e+00, 0.0000000e+00, 4.3000000e+01,
        2.0000000e+00, 1.2551082e+05, 1.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 7.9084100e+04]])

In [42]:
y[:5]

array([1, 0, 1, 0, 0])

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

### We'll scale the columns so the model can perform better.  We'll scale on the train and use that object to scale the unseen/test set.

In [44]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [51]:
cf = Sequential()
cf.add(Dense(output_dim=6, init='uniform', activation='relu', input_dim=10))
cf.add(Dense(output_dim=6, init='uniform', activation='relu'))
cf.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

In [52]:
cf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [55]:
cf.fit(X_train, y_train, nb_epoch=50, batch_size=20)

Epoch 1/50
7500/7500 [==============================] - 0s 55us/step - loss: 0.4069 - acc: 0.8333
Epoch 2/50
7500/7500 [==============================] - 0s 49us/step - loss: 0.4068 - acc: 0.8333: 0s - loss: 0.4057 - acc: 0.8
Epoch 3/50
7500/7500 [==============================] - 0s 52us/step - loss: 0.4062 - acc: 0.8340
Epoch 4/50
7500/7500 [==============================] - 0s 49us/step - loss: 0.4057 - acc: 0.8347
Epoch 5/50
7500/7500 [==============================] - 0s 48us/step - loss: 0.4056 - acc: 0.8356
Epoch 6/50
7500/7500 [==============================] - 0s 49us/step - loss: 0.4056 - acc: 0.8336
Epoch 7/50
7500/7500 [==============================] - 0s 55us/step - loss: 0.4047 - acc: 0.8352
Epoch 8/50
7500/7500 [==============================] - 0s 48us/step - loss: 0.4043 - acc: 0.8337
Epoch 9/50
7500/7500 [==============================] - 0s 50us/step - loss: 0.4042 - acc: 0.8336
Epoch 10/50
7500/7500 [==============================] - 0s 49us/step - loss: 0.4031 - a

In [56]:
scores = cf.evaluate(X, y)
print('%s: %.2f%%' % (cf.metrics_names[1], scores[1]*100))

10000/10000 [==============================] - 0s 13us/step
acc: 79.63%


In [57]:
y_pred = cf.predict(X_test)

In [58]:
cm = confusion_matrix(y_test, y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets